### Set up boto3

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from decouple import config

In [ ]:
db = boto3.resource(
    'dynamodb',
    aws_access_key_id = config('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = config('AWS_SECRET_ACCESS_KEY'),
    region_name = 'us-east-1'
    )
table = db.Table('dydxV3') # partition key: 'name'

In [ ]:
# test get
response = table.get_item(
    Key={
        'name': 'test'
    }
)
response['Item']

### Get prices from dydxV3

In [ ]:
from func_public import get_candles_historical
from func_connections import connect_dydx

In [ ]:
client = connect_dydx()

In [ ]:
re = get_candles_historical(client, 'MATIC-USD') 
re
# [{'datetime': '2024-02-28T00:00:00.000Z', 'BTC-USD': '57038'},..(old to new)]

In [ ]:
len(re)

In [ ]:
# build a list of prices
market = 'MATIC-USD'
prices = [] # old to new
for i in re:
    price = i[market]
    prices.append(price)
    
print(prices)
print(len(prices))

In [ ]:
dict = {market: prices}
print(dict)

In [ ]:
markets = ['BTC-USD', 'ETH-USD']
item_name = 'all_markets_close_prices'

In [ ]:
def update_markets_prices_dynamoBD(client, db_table, item_name):
    # DELETE item
    res_delete = table.delete_item(
        Key={
            'name': item_name
    })
    if res_delete['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f'DynamoDB: DELETE {item_name} failed')
    
    # Structure item
    ## Get all tradeable markets
    tradeable_markets = []
    markets = client.public.get_markets()

    ### Find "tradeable" pairs -> "ONLINE" & "PERPETUAL"
    for market in markets.data["markets"].keys():
        market_info = markets.data["markets"][market]
        if market_info["status"] == "ONLINE" and market_info["type"] == "PERPETUAL":
            tradeable_markets.append(market)
    
    ## Get close prices for each tradeable market
    item_dict = {}
    close_prices = []
    latest_datetime = ''
    timeframe = ''
    for market in tradeable_markets:
        candles = get_candles_historical(client, market)
        latest_datetime = candles[-1]['datetime']
        timeframe = str(len(candles))
        prices = []
        for c in candles:
            prices.append(c[market])
        close_prices.append({market: prices})
    item_dict = {
        'name': item_name,
        'latest_datetime': latest_datetime,
        'interval': '1hr',
        'timeframe': timeframe,
        'close_prices': close_prices
    }
    
    # PUT item
    res_put = db_table.put_item(Item=item_dict)
    if res_put['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f'DynamoDB: PUT {item_name} failed')
    
    print(f'{item_name} updated in DynamoDB')

In [ ]:
update_markets_prices_dynamoBD(client, table, item_name)